In [21]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
import random
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
import matplotlib.gridspec as gridspec
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut
from sklearn.neighbors import KNeighborsClassifier
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline



# Plotting style
sns.set_style('darkgrid')
sns.set_theme(font_scale=1.)

# Load dataset
df = pd.read_csv("../data/ObesityDataSet_Clean.csv")  

# Quick look at dataset
df.head()

target = "Weight"
feature_cols = ["Age", "Height", "FCVC", "NCP", "CH2O", "FAF", "TUE"]

# define target and features (just to extract y)
y = df[target].to_numpy()
print(y)

# baseline ignores X, but we still need to split the same way
X = df[feature_cols].copy()
print(X)


[ 64.   56.   77.  ... 133.7 133.3 133.5]
      Age  Height  FCVC  NCP  CH2O  FAF  TUE
0      21    1.62     2    3     2    0    1
1      21    1.52     3    3     3    3    0
2      23    1.80     2    3     2    2    1
3      27    1.80     3    3     2    2    0
4      22    1.78     2    1     2    0    0
...   ...     ...   ...  ...   ...  ...  ...
2106   21    1.71     3    3     2    2    1
2107   22    1.75     3    3     2    1    1
2108   23    1.75     3    3     2    1    1
2109   24    1.74     3    3     3    1    1
2110   24    1.74     3    3     3    1    1

[2111 rows x 7 columns]


*TWO LAYER CROSS VALIDATION*

In [28]:

def cross_validation(X, y, outer_folds=10, inner_folds=10, random_state=42):

    if not isinstance(y, pd.Series):
        y = pd.Series(y)

    lambda_grid = [1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
    rows = []
    CV_outer = KFold(n_splits=outer_folds, shuffle=True, random_state=random_state)
    outer_test_mse = []
    CV_inner = KFold(n_splits=inner_folds, shuffle=True, random_state=random_state)
    inner_mse = []


    for outer_fold_idx, (outer_train_idx, outer_test_idx) in enumerate(CV_outer.split(X), start=1):
        X_train_outer, X_test_outer = X.iloc[outer_train_idx], X.iloc[outer_test_idx]
        y_train_outer, y_test_outer = y.iloc[outer_train_idx], y.iloc[outer_test_idx]

        #standerdize data 
        scaler = StandardScaler()
        X_train_outer = pd.DataFrame(scaler.fit_transform(X_train_outer), columns=X_train_outer.columns)
        X_test_outer = pd.DataFrame(scaler.transform(X_test_outer), columns=X_test_outer.columns)
    

        lambda_errors = {}
        for lam in lambda_grid:
            inner_mses = []
            for inner_train_idx, inner_test_idx in CV_inner.split(X_train_outer):
                X_train_inner = X_train_outer.iloc[inner_train_idx]
                X_test_inner = X_train_outer.iloc[inner_test_idx]
                y_train_inner = y_train_outer.iloc[inner_train_idx]
                y_test_inner = y_train_outer.iloc[inner_test_idx]

                # LINEAR REGRESSION MODEL
                model = Ridge(alpha=lam)
                model.fit(X_train_inner, y_train_inner)
                y_pred_inner = model.predict(X_test_inner)
                inner_mse.append(mean_squared_error(y_test_inner, y_pred_inner))
                E_test_lin = mean_squared_error(y_test_outer,  model.predict(X_test_outer))
                inner_mses.append(mean_squared_error(y_test_inner, y_pred_inner))
                
            lambda_errors[lam] = np.mean(inner_mses)

        best_lambda = min(lambda_errors, key=lambda_errors.get)

        # TRAIN FINAL MODEL ON FULL TRAINING SET WITH BEST LAMBDA
        model = Ridge(alpha=best_lambda)
        model.fit(X_train_outer, y_train_outer)
        E_test_lin = mean_squared_error(y_test_outer, model.predict(X_test_outer))
        outer_test_mse.append(E_test_lin)

        # BASELINE MODEL
        baseline_pred = float(np.mean(y_train_outer))       
        E_test_base = mean_squared_error(y_test_outer, np.full(len(y_test_outer), baseline_pred))

        # ANN MODEL 
        lr = 0.01
        n_epochs = 1000 
        input_dim  = X_train_outer.shape[1]  # Number of features
        hidden_dim = 2
        output_dim = 1
        hyperparameters_to_tune = [1, 2, 3, 4, 5]
        best_h, best_val = None, np.inf
        criterion = torch.nn.MSELoss()

        def get_model(input_dim, hidden_dim, output_dim):
            return torch.nn.Sequential(
                torch.nn.Linear(in_features=input_dim, out_features=hidden_dim, bias=True),     # Input layer
                torch.nn.Tanh(),                                                                # Activation function
                torch.nn.Linear(in_features=hidden_dim, out_features=output_dim, bias=True),    # Output layer
            )

        # Loop over the hyperparameter settings        
        for hidden_dim in hyperparameters_to_tune:
            fold_vals = []
            for inner_train_idx, inner_test_idx in CV_inner.split(X_train_outer):
                X_train_inner = X_train_outer.iloc[inner_train_idx]
                X_test_inner = X_train_outer.iloc[inner_test_idx]
                y_train_inner = y_train_outer.iloc[inner_train_idx]
                y_test_inner = y_train_outer.iloc[inner_test_idx]

                X_train_inner_tensor = torch.tensor(X_train_inner.values, dtype=torch.float32)
                y_train_inner_tensor = torch.tensor(y_train_inner.values, dtype=torch.float32).view(-1, 1)
                X_test_inner_tensor  = torch.tensor(X_test_inner.values, dtype=torch.float32)
                y_test_inner_tensor  = torch.tensor(y_test_inner.values, dtype=torch.float32).view(-1, 1)
            
                model = get_model(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)
                optimizer = torch.optim.SGD(params=model.parameters(), lr=lr)
            
                for _ in range(n_epochs):
                    model.train()
                    outputs = model(X_train_inner_tensor)
                    loss = criterion(outputs, y_train_inner_tensor)
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                with torch.no_grad():
                    model.eval()
                    fold_vals.append(criterion(model(X_test_inner_tensor), y_test_inner_tensor).item())

            val_loss = np.mean(fold_vals)
            if val_loss < best_val:
                best_val = val_loss
                best_h = hidden_dim

        # TRAIN FINAL MODEL ON FULL TRAINING SET WITH BEST HYPERPARAMETER
        X_train_outer_tensor = torch.tensor(X_train_outer.values.astype(np.float32))
        y_train_outer_tensor = torch.tensor(y_train_outer.values.astype(np.float32).reshape(-1, 1))
        X_test_outer_tensor  = torch.tensor(X_test_outer.values.astype(np.float32))

        model = get_model(input_dim, best_h, output_dim)
        optimizer = torch.optim.SGD(model.parameters(), lr=lr)
        for _ in range(n_epochs):
            model.train()
            optimizer.zero_grad()
            loss = criterion(model(X_train_outer_tensor), y_train_outer_tensor)
            loss.backward()
            optimizer.step()

        with torch.no_grad():
            model.eval()
            E_test_ann = mean_squared_error(y_test_outer.values, model(X_test_outer_tensor).detach().numpy().ravel())

        rows.append({"Outer fold": outer_fold_idx,  "λ*_i": best_lambda, "E_test^lin": E_test_lin, "E_test^base": E_test_base,  "h*_i": best_h, "E_test^ann": E_test_ann})

    results = pd.DataFrame(rows).sort_values("Outer fold").reset_index(drop=True)
    print(results.round({"E_test^lin":2, "E_test^base":2, "E_test^ann":2}))
    print("\nAverages (MSE ↓) across outer folds:")
    print(f"Linear regression: {results['E_test^lin'].mean():.2f}")
    print(f"Baseline:          {results['E_test^base'].mean():.2f}")
    print(f"Neural Network:    {results['E_test^ann'].mean():.2f}")

    return results
results  = cross_validation(X, y, outer_folds=10, inner_folds=10, random_state=42)


   Outer fold  λ*_i  E_test^lin  E_test^base  h*_i  E_test^ann
0           1    10      462.83       732.32     5      377.56
1           2    10      439.68       677.76     5      345.52
2           3    10      477.38       736.37     5      428.83
3           4    10      370.21       614.22     5      349.21
4           5    10      460.64       634.78     5      397.01
5           6    10      475.32       685.18     5      368.67
6           7    10      409.38       614.44     5      332.20
7           8    10      424.39       716.06     5      336.35
8           9    10      526.75       786.46     5      370.54
9          10    10      452.80       667.27     5      358.86

Averages (MSE ↓) across outer folds:
Linear regression: 449.94
Baseline:          686.49
Neural Network:    366.48


*STATS EVALUATION*

In [32]:
from scipy import stats

# example: linear vs baseline
d_lin_base = results["E_test^lin"] - results["E_test^base"]
t_stat, p_val = stats.ttest_rel(results["E_test^lin"], results["E_test^base"])

mean_diff = d_lin_base.mean()
std_diff  = d_lin_base.std(ddof=1)
n = len(d_lin_base)
ci_low  = mean_diff - 2.262 * std_diff / np.sqrt(n)
ci_high = mean_diff + 2.262 * std_diff / np.sqrt(n)

print(f"Linear vs Baseline:")
print(f"mean diff = {mean_diff:.3f}")
print(f"t = {t_stat:.3f}, p = {p_val}")
print(f"95% CI = [{ci_low:.3f}, {ci_high:.3f}]")

Linear vs Baseline:
mean diff = -236.551
t = -21.103, p = 5.651823060668685e-09
95% CI = [-261.907, -211.196]


In [37]:
from scipy import stats

# example: linear vs ann
d_lin_base = results["E_test^ann"] - results["E_test^lin"]
t_stat, p_val = stats.ttest_rel(results["E_test^ann"], results["E_test^lin"])

mean_diff = d_lin_base.mean()
std_diff  = d_lin_base.std(ddof=1)
n = len(d_lin_base)
ci_low  = mean_diff - 2.262 * std_diff / np.sqrt(n)
ci_high = mean_diff + 2.262 * std_diff / np.sqrt(n)

print(f"Linear vs ANN:")
print(f"mean diff = {mean_diff:.3f}")
print(f"t = {t_stat:.3f}, p = {p_val}")
print(f"95% CI = [{ci_low:.3f}, {ci_high:.3f}]")

Linear vs ANN:
mean diff = -83.461
t = -7.341, p = 4.3670510870121584e-05
95% CI = [-109.176, -57.745]


In [34]:
from scipy import stats

# example: ann vs baseline
d_ann_base = results["E_test^ann"] - results["E_test^base"]
t_stat, p_val = stats.ttest_rel(results["E_test^ann"], results["E_test^base"])

mean_diff = d_lin_base.mean()
std_diff  = d_lin_base.std(ddof=1)
n = len(d_lin_base)
ci_low  = mean_diff - 2.262 * std_diff / np.sqrt(n)
ci_high = mean_diff + 2.262 * std_diff / np.sqrt(n)

print(f"Linear vs ANN:")
print(f"mean diff = {mean_diff:.3f}")
print(f"t = {t_stat:.3f}, p = {p_val}")
print(f"95% CI = [{ci_low:.3f}, {ci_high:.3f}]")

Linear vs ANN:
mean diff = 83.461
t = -18.961, p = 1.4529217140698072e-08
95% CI = [57.745, 109.176]
